# Genesis4 Benchmark on Perlmutter (NERSC)

HPC systems often have special commands to invoke MPI jobs. NERSC uses slurm, which provides `srun` in place of `mpirun`.

In [1]:
import os

from genesis.version4 import Genesis4

The `MPI_RUN` class attribute needs to be replaced. For convenience, Genesis4 tries to detect NERSC:

In [2]:
Genesis4.MPI_RUN

'"/Users/klauer/miniforge3/envs/lume-genesis/bin/mpirun" -n {nproc} {command_mpi}'

Additionally, the working directory must be on the scratch disk. This is automatically detected:

In [3]:
Genesis4.WORKDIR

Create Genesis4 object:

In [4]:
G = Genesis4("data/basic4/cu_hxr.in")

Set up the benchmark

In [7]:
G.input.main.track.zstop = 40

Run with MPI

In [8]:
%%time
G.verbose = False
G.nproc = 0
G.run()

CPU times: user 75.4 ms, sys: 42.2 ms, total: 118 ms
Wall time: 50 s


Genesis4Output(
  run=RunInfo(
    error_reason='',
    run_script='/Users/klauer/miniforge3/envs/lume-genesis/bin/mpirun -n 0 /Users/klauer/miniforge3/envs/lume-genesis/bin/genesis4 -l hxr.lat genesis4...
    output_log='---------------------------------------------\nGENESIS - Version 4.6.6 has started...\nCompile info: Compiled by runner at 2024-01-11 ...
    start_time=1588289.1093675,
    end_time=1588338.959887,
    run_time=49.850519499974325,
  ),
  beam=OutputBeam(
    units={
      'alphax': pmd_unit('rad', 1, (0, 0, 0, 0, 0, 0, 0)),
      'alphay': pmd_unit('rad', 1, (0, 0, 0, 0, 0, 0, 0)),
      'betax': pmd_unit('m', 1, (1, 0, 0, 0, 0, 0, 0)),
      'betay': pmd_unit('m', 1, (1, 0, 0, 0, 0, 0, 0)),
      'bunchingphase': pmd_unit('rad', 1, (0, 0, 0, 0, 0, 0, 0)),
      'current': pmd_unit('A', 1, (0, 0, 0, 1, 0, 0, 0)),
      'efield': pmd_unit('eV/m', 1.602176634e-19, (1, 1, -2, 0, 0, 0, 0)),
      'emax': pmd_unit('m_ec^2', 8.187105776885341e-14, (2, 1, -2, 0, 0, 0, 0)),
      'emin': pmd_unit('m_ec^2', 8.187105776885341e-14, (2, 1, -2, 0, 0, 0, 0)),
      'emitx': pmd_unit('m', 1, (1, 0, 0, 0, 0, 0, 0)),
      'emity': pmd_unit('m', 1, (1, 0, 0, 0, 0, 0, 0)),
      'energy': pmd_unit('m_ec^2', 8.187105776885341e-14, (2, 1, -2, 0, 0, 0, 0)),
      'energyspread': pmd_unit('m_ec^2', 8.187105776885341e-14, (2, 1, -2, 0, 0, 0, 0)),
      'lsc_field': pmd_unit('eV/m', 1.602176634e-19, (1, 1, -2, 0, 0, 0, 0)),
      'pxmax': pmd_unit('rad', 1, (0, 0, 0, 0, 0, 0, 0)),
      'pxmin': pmd_unit('rad', 1, (0, 0, 0, 0, 0, 0, 0)),
      'pxposition': pmd_unit('rad', 1, (0, 0, 0, 0, 0, 0, 0)),
      'pymax': pmd_unit('rad', 1, (0, 0, 0, 0, 0, 0, 0)),
      'pymin': pmd_unit('rad', 1, (0, 0, 0, 0, 0, 0, 0)),
      'pyposition': pmd_unit('rad', 1, (0, 0, 0, 0, 0, 0, 0)),
      'ssc_field': pmd_unit('eV/m', 1.602176634e-19, (1, 1, -2, 0, 0, 0, 0)),
      'wakefield': pmd_unit('eV/m', 1.602176634e-19, (1, 1, -2, 0, 0, 0, 0)),
      'xmax': pmd_unit('m', 1, (1, 0, 0, 0, 0, 0, 0)),
      'xmin': pmd_unit('m', 1, (1, 0, 0, 0, 0, 0, 0)),
      'xposition': pmd_unit('m', 1, (1, 0, 0, 0, 0, 0, 0)),
      'xsize': pmd_unit('m', 1, (1, 0, 0, 0, 0, 0, 0)),
      'ymax': pmd_unit('m', 1, (1, 0, 0, 0, 0, 0, 0)),
      'ymin': pmd_unit('m', 1, (1, 0, 0, 0, 0, 0, 0)),
      'yposition': pmd_unit('m', 1, (1, 0, 0, 0, 0, 0, 0)),
      'ysize': pmd_unit('m', 1, (1, 0, 0, 0, 0, 0, 0)),
    },
    globals=OutputBeamGlobal(
      units={},
      energy=array([], dtype=float64),
      energyspread=array([], dtype=float64),
      xposition=array([], dtype=float64),
      yposition=array([], dtype=float64),
      xsize=array([], dtype=float64),
      ysize=array([], dtype=float64),
    ),
    bunching=array(shape=(1084, 552), dtype=float64),
    bunchingphase=array(shape=(1084, 552), dtype=float64),
    energy=array(shape=(1084, 552), dtype=float64),
    energyspread=array(shape=(1084, 552), dtype=float64),
    xsize=array(shape=(1084, 552), dtype=float64),
    ysize=array(shape=(1084, 552), dtype=float64),
    lsc_field=array(shape=(1084, 552), dtype=float64),
    ssc_field=array(shape=(1084, 552), dtype=float64),
    efield=array(shape=(1084, 552), dtype=float64),
    wakefield=array(shape=(1084, 552), dtype=float64),
    emin=array(shape=(1084, 552), dtype=float64),
    emax=array(shape=(1084, 552), dtype=float64),
    pxmin=array(shape=(1084, 552), dtype=float64),
    pxmax=array(shape=(1084, 552), dtype=float64),
    pymin=array(shape=(1084, 552), dtype=float64),
    pymax=array(shape=(1084, 552), dtype=float64),
    pxposition=array(shape=(1084, 552), dtype=float64),
    pyposition=array(shape=(1084, 552), dtype=float64),
    xmin=array(shape=(1084, 552), dtype=float64),
    xmax=array(shape=(1084, 552), dtype=float64),
    ymin=array(shape=(1084, 552), dtype=float64),
    ymax=array(shape=(1084, 552), dtype=float64),
    xposition=array(shape=(1084, 552), dtype=float64),
    yposition=array(shape=(1084, 552), dtype=float64),
    alphax=array(sh

# Timing

In [ ]:
MAX_CPUS = os.cpu_count()
CPUS_PER_NODE = 128
MIN_CPUS = MAX_CPUS // 8
MAX_CPUS, MIN_CPUS

In [ ]:
def time1(nproc):
    G.verbose = False
    G.nproc = nproc
    G.nnode = (nproc - 1) // CPUS_PER_NODE + 1
    G.run()
    return G.output.run.run_time


time1(MAX_CPUS)

In [ ]:
%%time
nlist = []
tlist = []
n_cpu = MAX_CPUS

while n_cpu > 0:
    n = n_cpu
    if n < MIN_CPUS:
        break
    nlist.append(n)
    dt = time1(n)
    tlist.append(dt)
    print(f"{n} cores, {dt:0.1f} s")
    n_cpu = n_cpu // 2

In [ ]:
nlist, tlist

# Some saved stats

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

%config InlineBackend.figure_format = 'retina'

# Perlmutter
nlist, tlist = (
    [256, 128, 64, 32, 16, 8, 4, 2, 1],
    [
        12.407618761062622,
        9.337806940078735,
        15.55810832977295,
        27.734387397766113,
        49.104663372039795,
        113.33639931678772,
        184.2123155593872,
        367.3742334842682,
        555.4603695869446,
    ],
)
nlist = np.array(nlist)[::-1]
tlist = np.array(tlist)[::-1]

tref = tlist[0]

# M1 Max
nlistmac, tlistmac = (
    [8, 4, 2, 1],
    [38.151074171066284, 75.41113114356995, 146.3512842655182, 285.11587405204773],
)
nlistmac = np.array(nlistmac)
tlistmac = np.array(tlistmac)


plt.plot(nlist, tlist / tref, marker=".", label="Genesis4 Perlmutter")
plt.plot(nlistmac, tlistmac / tref, marker=".", label="Genesis4 M1 Max")
plt.plot(nlist, 1 / (nlist / nlist[0]), "--", label="ideal")
plt.xscale("log")
plt.yscale("log")
plt.xlabel("n cores")
plt.ylabel("run time (normalized)")
plt.legend()